## Import modules, save data to Pandas dataframe

In [1]:
from __future__ import division, print_function

from modules import PerformParser as parser
import numpy as np
import pandas as pd

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import os
np.set_printoptions(precision=2)

In [2]:
init_notebook_mode(connected=True)

# Lets look at the variables stored in our dataframe

In [3]:
datafile = pd.read_pickle('data/calibration.pickle')

In [4]:
list(datafile.columns)

['viewPos_XYZ',
 'viewMat_4x4',
 'trialNumber',
 'blockNumber',
 'IOD',
 'IPD',
 'ballPos_XYZ',
 'cycEyeInHead_XYZ',
 'leftEyeInHead_XYZ',
 'rightEyeInHead_XYZ',
 'leftEyePos_XYZ',
 'rightEyePos_XYZ',
 'calibrationPos_XYZ',
 'calibrationCounter',
 'eyeTimeStamp',
 'frameTime',
 'eventFlag']

In [5]:
datafile['cycEyeInHead_XYZ'] = datafile['cycEyeInHead_XYZ'].apply(lambda row: [-row[0],row[1],row[2]])

In [6]:
# def drawVector(row):

#     #viewPos_XYZ = row['viewPos_XYZ']
#     eihDir_xyz = row['cycEyeInHead_XYZ']
#     #cycGIWEndpoint_XYZ = np.add(viewPos_XYZ, + np.multiply(cycGIW_XYZ,5))

#     calibrationPos_XYZ = row['calibrationPos_XYZ']
    
#     eihDir = go.Scatter3d(x=[0,eihDir_xyz[0]*5],
#         y=[0,eihDir_xyz[2]*5],
#         z=[0,eihDir_xyz[1]*5],
#         mode='lines',
#         line = dict(
#           color=('rgb(20, 0, 145)'),
#           width = 2)
#         )
    
#     headPos = go.Scatter3d(x=[0],
#                            y=[0],
#                            z=[0],
#                            mode='markers',
#                            marker={'color': '#48186a', 'size': 10},
#                           )
    
#     calibPos = go.Scatter3d(x=[calibrationPos_XYZ[0]],
#                            y=[calibrationPos_XYZ[2]],
#                            z=[calibrationPos_XYZ[1]],
#                            mode='markers',
#                            marker={'color': 'rgb(0,144,144)', 'size': 5},
#                           )
    
#     return([headPos,eihDir,calibPos])

In [7]:
# allTraces = []

# for i in range(1000,1009):

#     calibPt = gbTrial.get_group(i)
#     traces = np.array(calibPt.apply(lambda row: drawVector(row),axis=1))
#     traces = [item for sublist in traces for item in sublist]
#     allTraces.extend(traces)


In [8]:

# # gbTrial = datafile.groupby('trialNumber')

# # calibPt = gbTrial.get_group(1002)
# # traces = np.array(calibPt.apply(lambda row: drawVector(row),axis=1))
# # traces = [item for sublist in traces for item in sublist]



# width = 800
# height = 800
# xRange = [-2,2]
# yRange = [-1,3]
# zRange = [-2,2]

# layout = go.Layout(title="Gaze in World", 
#                 width=width,
#                 height=height,
#                 showlegend=False,
#                 scene=go.Scene(aspectmode='manual',
#                             aspectratio=dict(x=1, y=1, z=1),
#                             xaxis=dict(range=xRange, title='x Axis'),
#                             yaxis=dict(range=yRange, title='y Axis'),
#                             zaxis=dict(range=zRange, title='z Axis'),

#                            ),
#                 margin=go.Margin(t=100),
#                 hovermode='closest',

#                 )


# fig=go.Figure(data=list(allTraces),layout=layout)


# iplot(fig)

A function that prepares the EIH ray for drawing in a plotly figure

In [9]:
def drawEIHRay(row):

    eihDir_xyz = row['cycEyeInHead_XYZ']
    
    eihDir = go.Scatter3d(x=[0,eihDir_xyz[0]*2],
        y=[0,eihDir_xyz[2]*2],
        z=[0,eihDir_xyz[1]*2],
        mode='lines',
        line = dict(
          color=('rgb(20, 0, 145)'),
          width = 2)
        )
    
    return eihDir


In [10]:
gbTrial = datafile.groupby('trialNumber')

allTraces = []
calibPt_pt = []

for i in range(1000,1009):
    
    calibPt = gbTrial.get_group(i)
    traces = np.array(calibPt.apply(lambda row: drawEIHRay(row),axis=1))
    allTraces.extend(traces)
    
    calibPt_pt.extend( gbTrial.get_group(i).head(1)['calibrationPos_XYZ'].values )


In [11]:
width = 800
height = 800
xRange = [-1,1]
yRange = [-1,3]
zRange = [-2,2]

calibPt_pt = np.array(calibPt_pt)
calibTrace = go.Scatter3d(x=calibPt_pt[:,0],
                           y=calibPt_pt[:,2],
                           z=calibPt_pt[:,1],
                           mode='markers',
                           marker={'color': 'rgb(0,144,144)', 'size': 5},
                          )

headTrace = go.Scatter3d(x=[0],y=[0],z=[0],
                       mode='markers',
                       marker={'color': '#48186a', 'size': 10})

allTraces.append(headTrace)
allTraces.append(calibTrace)

layout = go.Layout(title="Calibration - EIH Space", 
                width=width,
                height=height,
                showlegend=False,
                scene=go.Scene(aspectmode='manual',
                               xaxis=dict(range=xRange, title='x Axis'),
                               yaxis=dict(range=yRange, title='y Axis'),
                               zaxis=dict(range=zRange, title='z Axis'),
                               aspectratio=dict(x=1, y=1, z=1),
                           ),
                margin=go.Margin(t=100),
                hovermode='closest',

                )


fig=go.Figure(data=list(allTraces),layout=layout)

iplot(fig)

### Lets get measures of angular difference 
$\theta = \arccos( \hat{eih} • \hat{cal})$

where eih is the normalized eye in head vector, and cal is the normalized direction from the eye to the calibration point

In [12]:
def calculateGazeError(row):

    eyeToPointDir_xyz = row['calibrationPos_XYZ'] / np.linalg.norm(row['calibrationPos_XYZ'])
    eihDir_xyz = row['cycEyeInHead_XYZ'] / np.linalg.norm(row['cycEyeInHead_XYZ'])
    
    if(sum(eihDir_xyz)==0):
        return (np.nan,np.nan,np.nan,np.nan,np.nan)

    # calculate angular difference along any direction
    calErr2D = np.rad2deg( np.arccos( np.vdot(eihDir_xyz,eyeToPointDir_xyz)) )
    
    # use trig to calculate component of error along azimuth and elevation
    
    # Azimuth
    # tan(alpha) = op/aj = atan(horiz/depth) = atan(x component/z component) 
    eihAzimuthDeg = np.rad2deg(np.arctan(eihDir_xyz[0]/eihDir_xyz[2]))
    calibAzimuthDeg = np.rad2deg(np.arctan(eyeToPointDir_xyz[0]/eyeToPointDir_xyz[2]))
    diffAzimuthDeg = eihAzimuthDeg - calibAzimuthDeg
    
    # Elevation
    # tan(alpha) = op/aj = atan(height/depth) = atan(y component/z component) 
    eihElevationDeg = np.rad2deg(np.arctan(eihDir_xyz[1]/eihDir_xyz[2]))
    calibElevationDeg = np.rad2deg(np.arctan(eyeToPointDir_xyz[1]/eyeToPointDir_xyz[2]))
    diffElevationDeg = eihElevationDeg - calibElevationDeg
    
    return (calErr2D,eihAzimuthDeg,eihElevationDeg,calibAzimuthDeg,calibElevationDeg)

 It will return an error for certain rows.  Can't avoid that!

In [13]:
out = datafile.apply( lambda row: calculateGazeError(row),axis=1)
(calErr2D,eihAzimuthDeg,eihElevationDeg,calibAzimuthDeg,calibElevationDeg) = zip(*out)

/Users/gjdiaz/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning:

invalid value encountered in true_divide



### Assign values to columns of the dataframe

In [14]:
datafile['eihAzimuthDeg'] = eihAzimuthDeg
datafile['eihElevationDeg'] = eihElevationDeg

datafile['calibAzimuthDeg'] = calibAzimuthDeg
datafile['calibElevationDeg'] = calibElevationDeg

datafile['calErr2D'] = calErr2D

Now, lets take the mean and standard deviation using the "aggregate function"

In [15]:
gbTrial = datafile.groupby('trialNumber')
calibStatsDF = gbTrial.aggregate([np.mean,np.std])

Lets have a look at the first grid of points

### Beautiful!  Lets plot it

In [16]:
width = 800
height = 800

pts = range(0,9)
# pts = range(9,18)
# pts = range(18,26)

labels = calibStatsDF[('calErr2D','mean')].iloc[pts].values.astype('|S4')
labels = [l + str('&deg;') for l in labels]

# The calibration points to show


calibPoints = go.Scatter(x=calibStatsDF[('calibAzimuthDeg','mean')].iloc[pts],
                         y=calibStatsDF[('calibElevationDeg','mean')].iloc[pts],
                         mode='markers+text',
                         marker={'color': 'rgb(0,0,0)', 'size': 10},
                         text = labels,
                         textposition='top',
                         textfont=dict(
                             family='Trebuchet MS',
                             size=15,
                             color=('rgba(0,0,0,0.5)')),
                        )



eihPoints = go.Scatter(x=calibStatsDF[('eihAzimuthDeg','mean')].iloc[pts],
                       y=calibStatsDF[('eihElevationDeg','mean')].iloc[pts],
                       mode='markers',
                       marker={'color': 'rgb(0,200,200)', 'size': 10},) 
#                        error_x=dict(
#                            type='data',
#                            array=calibStatsDF[('eyeAzimuthDeg','std')][0:9],
#                            visible=True
#                        ),
#                        error_y=dict(
#                            type='data',
#                            array=calibStatsDF[('eyeElevationDeg','std')][0:9],
#                            visible=True
#                        )
#                      )
                       



layout = go.Layout(title='Calibration - Head Centered Angular space',
                   width=width,
                   height=height,
                   showlegend=False,
                   xaxis=dict(range=[-22,22], title='degrees azimuth'),
                   yaxis=dict(range=[-22,22], title='degrees elevation'),
                   scene=go.Scene(aspectmode='manual',
                                  aspectratio=dict(x=1, y=1),
                                 ),
                   margin=go.Margin(t=100),
                   hovermode='closest')

data = [calibPoints,eihPoints]
fig=go.Figure(data=data,layout=layout)

iplot(fig)